In [3]:
import pandas as pd
import pickle 
import json
import cv2
import glob
import matplotlib.pyplot as plt
import TransformHelper

In [4]:
def save_to_disk(data, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(data, file)
def load_from_disk(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
        return data
def read_normalized_json_to_df(filepath):
    full_file_df = ""
    with open(filepath, 'r', encoding="utf-8") as json_file:
        json_full = json.load(json_file)
    full_file_df = pd.json_normalize(json_full)
    return full_file_df

In [5]:
# Load transformation info 
transform_infos = load_from_disk("./map_image_transform_infos.pickle")
transform_matrix = transform_infos["perspective_transform_matrix"]
image_size = transform_infos["image_size"]

# Directories
single_data_path = "./pre/local_data/Single/"
dyadic_leader_data_path = "./pre/local_data/Dyadic_Leader/"
dyadic_follower_data_path = "./pre/local_data/Dyadic_Follower/"
single_files = glob.glob(single_data_path + "*_EyeTrackingData_*.json")
dyadic_leader_files = glob.glob(dyadic_leader_data_path + "*_EyeTrackingData_*.json")
dyadic_follower_files = glob.glob(dyadic_follower_data_path + "*_EyeTrackingData_*.json")

# Collider names of exploration sessions 
exploration_collider_names = "./pre/all_unique_colliders_names_1023_exploration.csv"


## Extract raw collider hits
From single & dyadic condition to get non-static collider names.

In [6]:
def extract_all_collider_hits(file_list):
    '''
    Extract all collider hits (by unique name and collider bounds center) from 
    a provided list of eye tracking formatted files. 
    '''

    full_df = pd.DataFrame()
    
    print("Processing files...")
    
    # Go through all files 
    for idx, one_file in enumerate(file_list):
        
        print(str(idx+1) + " of " + str(len(file_list)))

        with open(one_file, 'r', encoding="utf-8") as json_file:
            
            # load relevant parts, collider hits
            json_full = json.load(json_file)
            df = pd.json_normalize(json_full["trials"][0]["dataPoints"])
            
            # unpack collider hits further 
            df = df[['rayCastHitsCombinedEyes']]
            df = pd.concat([pd.json_normalize(row["rayCastHitsCombinedEyes"]) for index, row in df.iterrows()]).reset_index(drop=True)

            # extract subject id
            df["participantId"] = int(json_full["trials"][0]["participantId"])

            if full_df.empty:
                full_df = df.copy(deep=True)
            else:
                full_df = full_df.append(df)
        
    
    # drop some duplicates 
    full_df.reset_index(drop=True, inplace=True)
    full_df.drop_duplicates(["hitObjectColliderName","hitObjectColliderBoundsCenter.x","hitObjectColliderBoundsCenter.y","hitObjectColliderBoundsCenter.z"],inplace=True)
    full_df.drop(columns=["hitPointOnObject.x","hitPointOnObject.y","hitPointOnObject.z"],inplace=True)
    full_df.reset_index(drop=True, inplace=True)
    
    return full_df
    


In [7]:
# Extract from raw files or load preprocessed files
read_raw = False

if read_raw:
    # Extract from raw eye tracking files 
    single_full_df = extract_all_collider_hits(single_files)
    single_full_df["session_type"] = "single"
    dyadic_leader_full_df = extract_all_collider_hits(dyadic_leader_files)
    dyadic_leader_full_df["session_type"] = "dyadic_leader"
    dyadic_follower_full_df = extract_all_collider_hits(dyadic_follower_files)
    dyadic_follower_full_df["session_type"] = "dyadic_follower"
else:
    # Load already processed files 
    single_full_df = pd.read_csv("./pre/hits_for_animate_colliders_from_single.csv")
    dyadic_leader_full_df = pd.read_csv("./pre/hits_for_animate_colliders_from_dyadic_leader.csv")
    dyadic_follower_full_df = pd.read_csv("./pre/hits_for_animate_colliders_from_dyadic_follower.csv")
    

In [8]:
# Save preprocessed files
if False:
    single_full_df.to_csv("./pre/hits_for_animate_colliders_from_single.csv",index=False)
    dyadic_leader_full_df.to_csv("./pre/hits_for_animate_colliders_from_dyadic_leader.csv",index=False)
    dyadic_follower_full_df.to_csv("./pre/hits_for_animate_colliders_from_dyadic_follower.csv",index=False)

## Compare exploration colliders to mixed session type colliders

In [9]:
# Generate dataframe with mixed session colliders 
full_df = single_full_df.copy(deep=True)
full_df = full_df.append(dyadic_leader_full_df)
full_df = full_df.append(dyadic_follower_full_df)
full_df.sort_values("hitObjectColliderName",inplace=True)
full_df.reset_index(drop=True, inplace=True)

# Save individual collider names 
full_df.drop_duplicates(["hitObjectColliderName"]).sort_values("hitObjectColliderName").to_csv("./pre/all_unique_colliders_names_different_sessions.csv",index=False)


In [10]:
# Load exploration colliders 
exploration_colliders_df = pd.read_csv(exploration_collider_names)

In [11]:
# Compare different session colliders vs exploration colliders

print("Exploration colliders not in mixed sessions: " + str(list(set(exploration_colliders_df["hitObjectColliderName"]) - set(full_df["hitObjectColliderName"]))))
print()
print("Mixed session colliders not in exploration sessions: " + str(list(set(full_df["hitObjectColliderName"]) - set(exploration_colliders_df["hitObjectColliderName"]))))
colliders_not_in_exp = list(set(full_df["hitObjectColliderName"]) - set(exploration_colliders_df["hitObjectColliderName"]))


Exploration colliders not in mixed sessions: ['Concrete_fence_v1_s2 (6)', 'SmallWall_9', 'Concrete_fence_v1_s2 (4)', 'terrain_B.003', 'Concrete_fence_v1_s2 (8)', 'Lamppost_v1 (32)', 'SmallWall_15', 'road_dense_housing.013', 'Broadleaf_Mobile (206)', 'Trash_v2 (1)', 'Signboard_menu_v2', 'Broadleaf_Mobile (189)', 'Graffity_02', 'Lamppost_v1 (67)', 'Broadleaf_Mobile (53)', 'Concrete_fence_v1_s2 (2)', 'Broadleaf_Mobile (12)', 'Graffity_38', 'Concrete_fence_v1_s2 (1)', 'SmallWall_3', 'Lamppost_v1 (45)', 'SmallWall_40', 'SmallWall_2', 'Broadleaf_Mobile (223)']

Mixed session colliders not in exploration sessions: ['finger_pinky_1_r', 'Lamppost_v1 (197)', 'DestinationZoneA0', 'Fence_17', 'Lamppost_v1 (85)', 'StartZoneA6', 'StartZoneB2', 'DestinationZoneB5', 'Broadleaf_Mobile (140)', 'GrayRoom2', 'Concrete_fence_v1_s2 (32)', 'DestinationTriggerZoneA6', 'DestinationZoneB9', 'Broadleaf_Mobile (132)', 'Concrete_fence_v1_s2 (11)', 'DestinationTriggerZoneB2', 'Broadleaf_Mobile (161)', 'StartZoneA9'

## Manually select collider names of hand, phone, avatar

In [12]:
# Generate list of non stationary colliders
exclude_names = ["lamppost","broadleaf_mobile","destinationtriggerzone","destinationzone","startzone","concrete_fence", \
                "tree_planters","smallwall","gayroom","grayroom","road_block","fence", \
                "dumpster","arabic_v1_ceiling","road.011","bench_v1","prefab_unity_grass","raods_s_fix", \
                "museum_v1","oil_tank","building_191"]
animate_colliders = [x for x in colliders_not_in_exp if not any([name in x.lower() for name in exclude_names])]
animate_colliders.sort()

In [13]:
# Generate target names
animate_colliders_targets = ["female_avatar","male_avatar","phone","hand","hand","hand","hand","hand","hand","hand","hand","hand","hand","hand","hand","hand","hand","hand"]
for idx, _ in enumerate(animate_colliders):
    print("Original collider name: " + animate_colliders[idx] + " Target name: " + animate_colliders_targets[idx])

Original collider name: Female_063_NoHands Target name: female_avatar
Original collider name: Male_074_NoHands Target name: male_avatar
Original collider name: PhoneCollider Target name: phone
Original collider name: Sphere Target name: hand
Original collider name: Sphere (1) Target name: hand
Original collider name: Sphere (2) Target name: hand
Original collider name: Sphere (3) Target name: hand
Original collider name: finger_index_0_r Target name: hand
Original collider name: finger_index_1_r Target name: hand
Original collider name: finger_index_2_r Target name: hand
Original collider name: finger_middle_0_r Target name: hand
Original collider name: finger_middle_1_r Target name: hand
Original collider name: finger_middle_2_r Target name: hand
Original collider name: finger_pinky_1_r Target name: hand
Original collider name: finger_pinky_2_r Target name: hand
Original collider name: finger_ring_1_r Target name: hand
Original collider name: finger_ring_2_r Target name: hand
Original

In [14]:
# Create dataframe to be saved 
animate_colliders_df = pd.DataFrame(columns=["source_collider_name","target_collider_name"])
for idx, _ in enumerate(animate_colliders):
    animate_colliders_df = animate_colliders_df.append({"source_collider_name":animate_colliders[idx],"target_collider_name":animate_colliders_targets[idx]},ignore_index=True)
animate_colliders_df

,source_collider_name,target_collider_name
0,Female_063_NoHands,female_avatar
1,Male_074_NoHands,male_avatar
2,PhoneCollider,phone
3,Sphere,hand
4,Sphere (1),hand
5,Sphere (2),hand
6,Sphere (3),hand
7,finger_index_0_r,hand
8,finger_index_1_r,hand
9,finger_index_2_r,hand


In [19]:
# Save 
if False:
    animate_colliders_df.to_csv("./animate_collider_list.csv",index=False)